# Summary
This problem can be represented as a graph of `numCourse` nodes, and `len(prerequisites)` directed edges. Where the edge will point from 0 to 1 if 1 depends on 0 (0 is a prereq of 1).

The core of the problem is just to detect whether a cycle exists in this graph. If a cycle doesn't exist, then we will be able to finish all courses.

We do so by creating a separate `visiting` set to keep track of what node we are currently checking prereqs on.

We will loop through all all the number of courses, then at each course, we first check if it's already currently being tracked in the `visiting` set. If it's being visited, we have to return False and terminate the entire search.

Then we check if this course is already completed or can be completed. This will be accomplished by checking whether its value in the hashmap is en empty list. This means that it has no prereqs anymore and is available to be completed (or already been completed).

Otherwise, we have then recursively check the prereqs of this "prereq" as well, which makes us re enter the search function again, recursively.

Once all the prereqs of the current course has been iterated over, we know that a cycle hasn't been found and therefore this course is successfully visited. We can then remove it from `visiting` set and set its value in the hashmap to be an empty list to signify that it's a course that's been taken.

If we are able to iterate through all `numCourse`, then we can return False. But if during which any of the `dfs` function call returns False we immediately will return False in the main loop function.

## Time Complexity
$O(V + E)$ where $V$ the number of courses will simply be `numCourse` and $E$ the number of edges will be the total number of prerequisites since each prereq is a directed edge. Because in hashmap construction we first loop through each course so $O(V)$ but then we append each prereq each at $O(1)$ so in total we get $O(1 \times E)$.

And in the loop calling `dfs` we have to visit every node (i.e. in the initial hashmap construction we do this twice) and trace through every single edge in order to verify that no cycle exists, which is also $O(V+E)$. It's not just $O(E)$ because if nodes are isolated we still have to check though them each which takes $O(V)$. And it's not $O(V \times E)$ because not every `dfs` call will take up the entire $O(E)$ times. Once a course is cleared from all prereqs it's set to an empty list. Then the future checks are $O(1)$ time. So in other word, each edge will only be visited once.

## Space Complexity
$O(V + E)$ for the size of `hashmap` ($V$ keys but also the values can be $E$), $O(V)$ for `visiting`. And the recursive stack can at most grow to the size of courses so $O(V)$ because the deepest stack would be a linear graph. The stack depth is the longest path in a graph without a cycle, which in the extreme case can be the total number of nodes. The total number of edges doesn't affect the stack depth. For example if four nodes all point to a same one node, each of the path will still only have a stack depth of 2.

Full transcription of time and space complexity analysis: https://claude.ai/public/artifacts/51bb4028-7924-44ae-a045-57bff8f416d7

In [ ]:
from typing import List


class Solution:
    def canFinish(self, numCourse: int, prerequisites: List[List[int]]) -> bool:
        hashmap = dict()
        visiting = set()

        for course in range(numCourse):
            hashmap[course] = []

        for prereq in prerequisites:
            hashmap[prereq[0]].append(prereq[1])

        def dfs(course):
            if course in visiting:
                return False
            if not hashmap[course]:
                return True
            
            visiting.add(course)
            for neighbor in hashmap[course]:
                if not dfs(neighbor):
                    return False
            
            visiting.remove(course)
            hashmap[course] = []

            return True

        for course in range(numCourse):  # we can't just trigger one and have everything recursively visited because there may be isolated courses; i.e. 1->2 and 3->4
            if not dfs(course):
                return False
        
        return True

# Summary

Loop through prerequisites, and establish a dictionary where the keys are the first element of each sub-list. And the prerequisites of each course will be the corresponding value which is a list of prerequisite courses.

Then we iterate the keys one by one. At each key, we loop through its prereq list. Then for each prereq, we also check its prereqs, and if all its prereqs have been taken, or that it doesn't exist in the requirement hashmap, then mark the original key as take

check whether it's been taken. If not, take it (visited) and decrement the numCourses variable by 1.

We break out if numCourses has reached 0.

At the end if we can successfully loop through everything, then we can return True.


## Time Complexity
* Building of the hashmap will take $O(E)$ where E is the number of prerequisites (length of the prereqs list)
* Then `check_eligibility` will be called maximally by the number of courses that exist, $O(V)$
* Each `check_eligibility` can at most then need to loop through all of prereqs again, because each prereq may have its own prereqs 

So the last two bullet points together will take $O(V+E)$

## Space Complexity
$O(V + E)$ because `taken`, and `processing` can each be at most $O(V)$. And for `hashmap` it will be $O(E)$ for each prereq



In [20]:
from typing import List


class SolutionDebug:
    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        hashmap = dict()
        taken = set()
        processing = set()

        for prereq_list in prerequisites:
            if prereq_list[0] not in hashmap:
                hashmap[prereq_list[0]] = prereq_list[1:]
            else:
                hashmap[prereq_list[0]].extend(prereq_list[1:])
        
        print(f"Hashmap = {hashmap}")

        def check_eligibility(prereqs, numCourses):
            for prereq in prereqs:
                if numCourses <= 0:
                    return False, numCourses
                if prereq in taken:
                    continue
                elif prereq in processing:
                    print(f"{prereq} already in processing={processing}")
                    return False, -1
                elif prereq not in hashmap:
                    taken.add(prereq)
                    numCourses -= 1
                elif prereq in hashmap:
                    processing.add(prereq)
                    print(f"Processing = {processing} to check eligibility")
                    eligibility, numCourses = check_eligibility(hashmap[prereq], numCourses)
                    if not eligibility:
                        return False, numCourses
                    processing.remove(prereq)
            return True, numCourses
        
        for course, prereqs in hashmap.items():
            processing.add(course)
            print(f"Processing = {processing} to check eligibility")
            eligibility, numCourses = check_eligibility(prereqs, numCourses) 
            if eligibility:
                taken.add(course)
                numCourses -= 1
            else:
                return False
            processing.remove(course)
        return True
            
                

In [21]:
prerequisites = [[1,4],[2,4],[3,1],[3,2]]
numCourses = 5

s = SolutionDebug()
s.canFinish(numCourses, prerequisites)

Hashmap = {1: [4], 2: [4], 3: [1, 2]}
Processing = {1} to check eligibility
Processing = {2} to check eligibility
Processing = {3} to check eligibility


True

Add all course to the hashmap, if a key maps to an empty list, we know that its available to take, then we can add it to `visited` and decrement `numCourses` by 1.

